## 한글 폰트 설치

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

## matplotlib 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용


## load dataset

In [1]:
import pandas as pd
df_FSSH = pd.read_csv('fruitstore_saleshistory.csv')
df_FSSH[:3]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05


### 연속형 데이터 정규성 검정

In [2]:
# stats.anderson을 사용한 정규 분포 검정
import scipy.stats as stats
result = stats.anderson(df_FSSH['구매금액'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


## 범주형 / 연속형 : 범주 대한 관계성 확인

### 연속형이 비정규 분포이며, 범주형 항목이 3개 이상일 떄

#### 예제 가설 : 요일, 구매금액(항목 2개 선택)

#### 분석 내용 : 요일(범주형)에 따른 고객의 구매 금액(연속형) 분산을 비교하여, 특정 요일에 구매 금액의 변동성이 큰지 분석합니다.
- 귀무 가설 : 요일별 고객의 구매 금액 분산이 이분산을 보이지 않는다.
- 대립 가설 : 요일별 고객의 구매 금액 분산이 이분산을 보인다.

#### 요일 별 구매금액 합계 확인

In [11]:
purchase_sum_by_day = df_FSSH.groupby('요일')['구매금액'].sum()

for day, sum in purchase_sum_by_day.items():
    print(f"요일이름: {day}, 구매금액 합계: {sum}")


요일이름: 금, 구매금액 합계: 1215650577
요일이름: 목, 구매금액 합계: 1130698629
요일이름: 수, 구매금액 합계: 1161404346
요일이름: 월, 구매금액 합계: 1305186165
요일이름: 일, 구매금액 합계: 591561808
요일이름: 토, 구매금액 합계: 1076943090
요일이름: 화, 구매금액 합계: 1223417575


## 정규성 검증 필요


### Kruskal Test
- 연속형이 비정규 분포 이면서 범주형에 따른 항목이 등분산, 항목 갯수가 3개 이상

In [25]:
import scipy.stats as stats

# 요일별로 구매 금액 데이터 추출
purchase_by_day_list = []
for day in df_FSSH['요일'].unique():
    day_df = df_FSSH[df_FSSH['요일'] == day]
    day_data = day_df.dropna(subset=['구매금액'])
    data = day_data['구매금액']
    purchase_by_day_list.append(data)

stats.kruskal(*purchase_by_day_list)

KruskalResult(statistic=105.6876345106788, pvalue=1.6278456573580912e-20)

In [33]:
round(1.6278456573580912e-20, 6)

0.0

### 분석결과
- 통계 결론 : 연속형 데이터의 정규성을 Anderson Test로 검사 했을 때 비졍규 분포를 띔. 범주형 데이터가 3개 이상이므로 Kruskal Test 시행. p-value 0.05 이하이므로 대립 가설 채택.
- 사실 결론 : 요일별 고객의 구매 금액 분산이 이분산을 보인다.